# Redes bayesianas - Resident Evil

En este trabajo usaremos una red bayesiana construida por nosotros. Esta red se basa en un contexto imaginario del mundo de __Resident Evil__. Esta pensada con el objetivo de que un superviviente pueda saber cuales son sus posibilidades de mantenerse con vida teniendo en cuenta los recursos y otras variables del universo.

Para la realización de la red se ha usado la herramienta __Weka__. 
Una vez construida, se ha exportado en formato .xml y retocado para que el método de pgmpy la lea correctamente.

El grafo de la red es el que se muestra a continuación:

<img src='G_RE.png' width=700 height=463>

Para poder trabajar con redes bayesianas usaremos el paquete __pgmpy__. Lo primero que haremos será importar el paquete y las herramientas necesarias.

In [137]:
#Importamos el paquete pgmpy y las herramientras necesarias

import networkx  # Permite trabajar con grafos
import pgmpy.models as pgmm  # Modelos gráficos de probabilidad
import pgmpy.factors as pgmf  # Tablas de probabilidades condicionales y
                              # potenciales de probabilidad
import pgmpy.inference as pgmi  # Inferencia probabilística exacta

#Librerías para cálculo de heurísticas
import operator
from itertools import combinations

Además, para leer ficheros con extensión __.XMLBIF__ necesitaremos importar la herramienta __readerwrite.XMLBIF__ de pgmpy.

In [138]:
import pgmpy.readwrite.XMLBIF as rwBIF

reader = rwBIF.XMLBIFReader('residentevil.xml')
Modelo_residentevil = reader.get_model()

Vamos a ver los __nodos, aristas__ y __tablas__ de la red:

In [139]:
#Nodos de la red
Modelo_residentevil.nodes()

['Lugar',
 'Ciudad',
 'Horda',
 'Medicina',
 'Agua',
 'Comida',
 'Refugio',
 'Vehiculo',
 'Armas',
 'Grupo',
 'Soldados umbrella',
 'Cooperativo']

In [140]:
#Aristas de la red
Modelo_residentevil.edges()

[('Lugar', 'Ciudad'),
 ('Lugar', 'Horda'),
 ('Ciudad', 'Horda'),
 ('Ciudad', 'Medicina'),
 ('Ciudad', 'Agua'),
 ('Ciudad', 'Comida'),
 ('Ciudad', 'Refugio'),
 ('Ciudad', 'Vehiculo'),
 ('Ciudad', 'Armas'),
 ('Ciudad', 'Grupo'),
 ('Ciudad', 'Soldados umbrella'),
 ('Refugio', 'Cooperativo'),
 ('Grupo', 'Cooperativo')]

In [141]:
#CPDS de la red
Modelo_residentevil.cpds

[<TabularCPD representing P(Lugar:3) at 0x118969f60>,
 <TabularCPD representing P(Ciudad:2 | Lugar:3) at 0x118969f98>,
 <TabularCPD representing P(Horda:2 | Ciudad:2, Lugar:3) at 0x118969390>,
 <TabularCPD representing P(Medicina:2 | Ciudad:2) at 0x118969630>,
 <TabularCPD representing P(Agua:2 | Ciudad:2) at 0x118969940>,
 <TabularCPD representing P(Comida:2 | Ciudad:2) at 0x118969e80>,
 <TabularCPD representing P(Refugio:2 | Ciudad:2) at 0x118969d68>,
 <TabularCPD representing P(Vehiculo:2 | Ciudad:2) at 0x118969d30>,
 <TabularCPD representing P(Armas:2 | Ciudad:2) at 0x1189694e0>,
 <TabularCPD representing P(Grupo:2 | Ciudad:2) at 0x1189695c0>,
 <TabularCPD representing P(Soldados umbrella:2 | Ciudad:2) at 0x1189697b8>,
 <TabularCPD representing P(Cooperativo:2 | Refugio:2, Grupo:2) at 0x118969860>]

Y por último, es necesario comprobar que la red no tiene errores de construcción. Para ello empleamos el método __check_model__ de pgmpy:

In [142]:
#Comprueba que el modelo no contiene errores, en caso de ser correcto devuelve True.
Modelo_residentevil.check_model()

/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


True

# Eliminación de variables

Una vez tenemos la red construida y cargada, vamos a diseñar el algoritmo de __eliminación de variables__. Se han seguido los siguientes pasos:

1. Descartar las variables irrelevantes para la consulta antes de comen- zar (mostrando un mensaje con la lista de variables descartadas).
2. Obtener todos los factores iniciales a partir de la red, ignorando las variables descartadas y teniendo en cuenta las evidencias.
3. Eliminar las variables ocultas, a través de las correspondientes ope- raciones sobre los factores.
4. Multiplicar los factores finales (en caso de que haya más de uno).
5. Normalizar.

La consulta utiliza como ejemplo durante el desarrollo del algoritmo ha sido: la probabilidad de que __Refugio__ sea __verdadero__, sabiendo que __Ciudad__ y __Cooperativo__ son __verdaderos__.

Finalmente, con el fin de comprobar la correcta implementación del algoritmo, se ha usado una herramienta __VariableElimination__ incluida en pgmpy que aplica directamente dicho algoritmo.

In [143]:
#Paso 1: Función que descarta las variables irrelevantes.

def descarta(m, c, e):
    lista = c+e
    descartados = []
    padres = []
    for l in lista:
        subgrafo = Modelo_residentevil.subgraph(networkx.ancestors(Modelo_residentevil, l))
        for s in subgrafo:
            padres.append(s)
                          
    for n in m.nodes():
        if n not in lista and n not in padres:
            descartados.append(n)
                  
    return descartados  

__Ejemplo 1:__ Probabilidad de Refugio sabiendo que si Ciudad y si Cooperativo. 

In [144]:
#Probando la función descartada

consulta = []
consulta.append("Refugio")
evidencias = []
evidencias.append("Ciudad")
evidencias.append("Cooperativo")

descarta(Modelo_residentevil,consulta,evidencias)

['Horda',
 'Medicina',
 'Agua',
 'Comida',
 'Vehiculo',
 'Armas',
 'Soldados umbrella']

In [145]:
#Paso 2: Función que devuelve los factores iniciales reducidos según las evidencias.

def factores_iniciales(m,d,e):
    c = 0
    factores = {}
    for n in m.nodes():
        if n not in d:
            factores[n]= m.cpds[c].to_factor()     
        c+=1
    for k,v in factores.items():
        for i in v.scope():
            if i in e:
                v = v.reduce([(i, e[i])])
        
    return factores

In [146]:
#Llamada a la función factores_iniciales
d = descarta(Modelo_residentevil, consulta, evidencias)
e = {'Ciudad':1, 'Cooperativo':1}
factores = factores_iniciales(Modelo_residentevil,d,e)

for clave, valor in factores.items():
    print(clave)
    print(valor.scope())
    print(valor)

Lugar
['Lugar']
╒═════════╤══════════════╕
│ Lugar   │   phi(Lugar) │
╞═════════╪══════════════╡
│ Lugar_0 │       0.3000 │
├─────────┼──────────────┤
│ Lugar_1 │       0.2000 │
├─────────┼──────────────┤
│ Lugar_2 │       0.5000 │
╘═════════╧══════════════╛
Ciudad
['Lugar']
╒═════════╤══════════════╕
│ Lugar   │   phi(Lugar) │
╞═════════╪══════════════╡
│ Lugar_0 │       0.4000 │
├─────────┼──────────────┤
│ Lugar_1 │       0.8000 │
├─────────┼──────────────┤
│ Lugar_2 │       0.2000 │
╘═════════╧══════════════╛
Refugio
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.4000 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6000 │
╘═══════════╧════════════════╛
Grupo
['Grupo']
╒═════════╤══════════════╕
│ Grupo   │   phi(Grupo) │
╞═════════╪══════════════╡
│ Grupo_0 │       0.2000 │
├─────────┼──────────────┤
│ Grupo_1 │       0.8000 │
╘═════════╧══════════════╛
Cooperativo
['Grupo', 'Refugio']
╒═════

In [147]:
#Paso 3: Función que elimina las variables ocultas.
def calcula_elimina(factores, consulta, evidencias):
    elimina = []
    for k,v in factores.items():
        if k not in consultas and k not in evidencias:
            elimina.append(k)
    return elimina

def eliminar_variables(factores, consultas, evidencias):
    elimina = calcula_elimina(factores,consultas,evidencias)
    res = {}
    
    for e in elimina:
        for k,v in factores.items():
            if e in v.scope() and len(v.scope()) > 1:
                phi_not = pgmf.factor_product(factores[e],v)
                phi_not.marginalize([e])
                factores[k] = phi_not
    for k,v in factores.items():
        for i in v.scope():
            if i in consultas:
                res[k] = v
        
    return res

In [148]:
#Llamada a la función eliminar_variables

consultas = []
consultas.append("Refugio")
evidencias = []
evidencias.append("Ciudad")
evidencias.append("Cooperativo")

d = descarta(Modelo_residentevil, consultas, evidencias)
e = {'Ciudad':1, 'Cooperativo':1}

factores = factores_iniciales(Modelo_residentevil,d,e)

ev = eliminar_variables(factores,consultas,evidencias)
          
for k,v in ev.items():
    print(k)
    print(v.scope())
    print(v)

Refugio
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.4000 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6000 │
╘═══════════╧════════════════╛
Cooperativo
['Refugio']
╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.8400 │
├───────────┼────────────────┤
│ Refugio_1 │         0.9200 │
╘═══════════╧════════════════╛


In [149]:
#Paso 4 y 5: Función que multiplica y normaliza los factores restantes.

def mn_factores(factores):
    m = []
    for k,v in factores.items():
        if not not v.scope():
            m.append(v)
        
    phi = pgmf.factor_product(*m)
    phi.normalize()
    
    return phi

In [150]:
#Llamada a la función mn_factores
end = mn_factores(eliminar_variables(factores,consultas,evidencias))
print(end)

╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.3784 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6216 │
╘═══════════╧════════════════╛


In [151]:
#Uso de la herramienta VariableElimination para comprobar el correcto resultado usando el ejemplo 1

Modelo_re_ev = pgmi.VariableElimination(Modelo_residentevil)
consulta = Modelo_re_ev.query(['Refugio'], {'Ciudad': 1, 'Cooperativo': 1})
print(consulta['Refugio'])

╒═══════════╤════════════════╕
│ Refugio   │   phi(Refugio) │
╞═══════════╪════════════════╡
│ Refugio_0 │         0.3784 │
├───────────┼────────────────┤
│ Refugio_1 │         0.6216 │
╘═══════════╧════════════════╛


/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


__Ejemplo 2:__ Probabilidad de Ciudad sabiendo que si Agua y no Refugio. 

In [152]:
# Construyendo el ejemplo 2

consultas.append("Ciudad")
evidencias = []
evidencias.append("Agua")
evidencias.append("Refugio")

d = descarta(Modelo_residentevil, consultas, evidencias)
e = {'Agua':1, 'Refugio':0}

factores = factores_iniciales(Modelo_residentevil,d,e)

ev = eliminar_variables(factores,consultas,evidencias)
end = mn_factores(eliminar_variables(factores,consultas,evidencias))
print(end)


╒══════════╤═══════════════╕
│ Ciudad   │   phi(Ciudad) │
╞══════════╪═══════════════╡
│ Ciudad_0 │        0.7099 │
├──────────┼───────────────┤
│ Ciudad_1 │        0.2901 │
╘══════════╧═══════════════╛


In [153]:
#Uso de la herramienta VariableElimination para comprobar el correcto resultado usando el ejemplo 2

Modelo_re_ev = pgmi.VariableElimination(Modelo_residentevil)
consulta = Modelo_re_ev.query(['Ciudad'], {'Agua': 1, 'Refugio': 0})
print(consulta['Ciudad'])

/Users/anapareciendo/anaconda/lib/python3.6/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


╒══════════╤═══════════════╕
│ Ciudad   │   phi(Ciudad) │
╞══════════╪═══════════════╡
│ Ciudad_0 │        0.7099 │
├──────────┼───────────────┤
│ Ciudad_1 │        0.2901 │
╘══════════╧═══════════════╛


# Heurísticas para el orden de eliminación

-__Min Degree__: dado un conjunto de factores, la variable que se decide eliminar es la que está conectada con menos variables en el conjunto de factores. Decimos que una variable está conectada con otra respecto de un conjunto de factores, cuando ambas aparecen en un mismo factor del conjunto.

-__Min Fill__: dado un conjunto de factores, la variable que se decide eliminar es aquella que al eliminarse introduciría menos conexiones nuevas (conexiones en el sentido explicado en el punto anterior).

-__Min Factor__: dado un conjunto de factores, la variable que se decide eliminar es aquella que al eliminarse produciría el factor más pequeño (es decir, el que menos entradas tendría).

In [154]:
def min_degree(factores, consultas, evidencias):
    res = {}
    elimina = calcula_elimina(factores, consultas, evidencias)
    cont = 0
    for e in elimina:
        for k,v in factores.items():
            if e in v.scope() and len(v.scope()) > 1:
                cont=cont+1
        res[e] =cont
        cont = 0
    resultado = sorted(res.items(), key=operator.itemgetter(1))
    return resultado
    
print("El orden de eliminación usando heurística Min Degree es:",
      min_degree(factores, consultas, evidencias))

El orden de eliminación usando heurística Min Degree es: [('Lugar', 0)]


In [155]:
#El coste de eliminar un nodo es el número de aristas que se añaden al eliminarlo
def min_fill(factores, consultas, evidencias):
    res = {}
    elimina = calcula_elimina(factores, consultas, evidencias)
    cont = 0
    
    for e in elimina:
        res[e] = len(list(combinations(Modelo_residentevil.neighbors(e), 2)))
    resultado = sorted(res.items(), key=operator.itemgetter(1))
    return resultado

print("El orden de eliminación usando heurística Min Fill es:",
      min_fill(factores, consultas, evidencias))

El orden de eliminación usando heurística Min Fill es: [('Lugar', 1)]


# Recomendador de evidencias

El funcionamiento del recomendador es el siguiente:

1. Se parte de una inferencia introducida por el usuario (con una variable de consulta y una o más de evidencia). En primer lugar el sistema deberá identificar qué variables de la red son “candidatas” a ser recomendadas-

2. Para cada una de dichas variables, y para cada uno de los valores de su correspondiente dominio, se deberá lanzar un algoritmo de inferencia aproximada añadiendo la evidencia en cuestión. (Se repetirán esas inferencias aproximadas varias veces para evitar ruido)

3. En base a los resultados obtenidos en el punto anterior, el sistema recomendará al usuario una variable.

In [156]:
import random

#Función que devuelve una probabilidad concreta del modelo
def seleccionar_probabilidad(cpd, valor, evidencia):
    # Buscamos los padres de una evidencia
    padres = cpd.evidence if cpd.evidence else []
    # Buscamos los valores asociados a los padres
    valores_evidencia = tuple(evidencia[var] for var in padres)
    # Buscamos los valores de la distribución de probabilidad conjunta
    return cpd.values[valor][valores_evidencia]

#Función que genera un valor aleatorio según la cardinalidad de la variable
def generar_valor_aleatorio(cardinalidad, probabilidades):
    p = random.random()
    acumuladas = 0
    for valor in range(cardinalidad):
        acumuladas += probabilidades[valor]
        if p <= acumuladas:
            return valor

In [157]:
#Función que genera una muestra aleatoria dada una red bayesiana
#Esta función usa las dos anteriores (seleccionar_probabilidad y generar_valor_aleatorio)
def muestra_aleatoria(red):
    variables = networkx.topological_sort(red)
    muestra = {}
    for variable in variables:
        tabla = Modelo_residentevil.get_cpds(variable)
        #TODO: con Lugar no funciona bien, hay que arreglarlo
        if variable != 'Lugar':
            cardinalidad = tabla.variable_card
            probabilidades = [seleccionar_probabilidad(tabla, i, muestra)
                            for i in range (cardinalidad)]
            valor = generar_valor_aleatorio(cardinalidad, probabilidades)
        else:
            valor = 0
        muestra[variable] = valor
        
    return muestra

In [158]:
#Probando la función anterior
muestra_aleatoria(Modelo_residentevil)

{'Agua': 0,
 'Armas': 1,
 'Ciudad': 1,
 'Comida': 0,
 'Cooperativo': 0,
 'Grupo': 1,
 'Horda': 0,
 'Lugar': 0,
 'Medicina': 0,
 'Refugio': 0,
 'Soldados umbrella': 1,
 'Vehiculo': 1}

El algoritmo de inferencia aproximada elegido para este trabajo a sido __Muestreo por rechazo__.
Con el generaremos un número de muestras (en nuestro caso 1000) de las que algunas no serán consistentes con las evidencias y otras si. Estas últimas serán las muestras válidas. De las muestras consistentes con las evidencias (las válidas) debemos distinguir las que tienen la variable de la consulta como un no y las que la tienen como un si.


In [159]:
import itertools

#Función que devuelve las frecuencias aplicando el algoritmo de muestreo por rechazo
def muestreo_con_rechazo(red, consulta, evidencia, N):
    muestras = [muestra_aleatoria(red) for i in range(N)] # Me genenará una muestra de N valores
    
    # Ahora nos quedamos con aquellas en las que la muestra y la evidencia coinciden
    muestrasValidas = []
    for muestra in muestras:
        if all(muestra[variable] == evidencia[variable]
              for variable in evidencia):
            muestrasValidas.append(muestra)
            
    cardinalidades = [red.get_cpds(variable).variable_card
                    for variable in consulta]
    
    # Calculamos las combinaciones 
    combinaciones = itertools.product(*(range (i)
                                       for i in cardinalidades))
    
    # Ahora contamos las combinaciones, es decir, el nº de veces que ocurre cada cosa
    # Recuerda: es un diccionario
    frecuencias = {combinacion : 0 
                   for combinacion in combinaciones}
    
    for muestra in muestrasValidas:
        combinacion = tuple(muestra[variable]
                       for variable in consulta)
        frecuencias[combinacion] += 1
    
    #print (combinaciones)
    #print (frecuencias)
    #print (len(muestras))
    #print (len(muestrasValidas))
    return frecuencias

In [160]:
#Probando la función anterior
random.seed(12345)
print('Las frecuencias son:',muestreo_con_rechazo(Modelo_residentevil, ['Refugio'], {'Ciudad': 1, 'Cooperativo': 1}, 10000))

Las frecuencias son: {(0,): 818, (1,): 2340}


Usaremos el algoritmo de muestreo por rechazo implementado anteriormente para probar con cada una de las variables __candidatas__ a ser recomendada, y finalmente __será ganaradora aquella que marque una diferencia mayor__ en el calculo de la consulta realizada.

In [161]:
#Función que usa el muestreo por rechazo para calcular la variable que nos aporta una probabilidad más acertada
def recomendador(modelo, consulta, evidencias):
    k = 0
    suma = 0
    i = 0
    candidatas = []
    e = list(evidencias.keys())
    nevidencias = {}
    res={}
    
    descartes = descarta(modelo, consulta, e)
    nodos = modelo.nodes()
    
    #Calculamos las candidatas
    for n in nodos:
        if (n not in consulta) and (n not in e) and (n not in descartes):
            candidatas.append(n)
   
    #Aplicamos muestreo por rechazo con las candidatas con valor 0
    for c in candidatas:
        nevidencias.update(evidencias)
        #Añadimos al diccionario de evidencias la candidata con valor 0
        nevidencias[c] = 0
        #Llamamos al algoritmo muestreo con rechazo
        frecuencias = muestreo_con_rechazo(modelo, consulta, nevidencias, 10000)
        #Normalizamos
        suma = frecuencias[(0,)] + frecuencias[(1,)]
        pno = frecuencias[(0,)] / suma
        psi = frecuencias[(1,)] / suma
        #Calculamos la diferencia
        dif = abs(pno-psi)
        #Asignamos la nueva candidata si la diferencia es mayor
        if(k < dif):
            k = dif
            i = 0
            r = c
        nevidencias.clear()
        
    #TODO: esto hay que arreglarlo, ya que Lugar no lo coge bien, tengo que borrarla
    candidatas.remove('Lugar')
    
    #Aplicamos muestreo por rechazo con las candidatas con valor 1
    for c in candidatas:
        nevidencias.update(evidencias)
        #Añadimos al diccionario de evidencias la candidata con valor 1
        nevidencias[c] = 1
        #Llamamos al algoritmo muestreo con rechazo
        frecuencias = muestreo_con_rechazo(modelo, consulta, nevidencias, 10000)
        #Normalizamos
        suma = frecuencias[(0,)] + frecuencias[(1,)]
        pno = frecuencias[(0,)] / suma
        psi = frecuencias[(1,)] / suma
        #Calculamos la diferencia
        dif = abs(pno-psi)
        #Asignamos la nueva candidata si la diferencia es mayor
        if(k < dif):
            k = dif
            i = 1
            r = c
        nevidencias.clear()
    
    #Devolvemos la variable candidata cuya diferencia es mayor como recomendación
    
    res[r] = i
    return res

__Ejemplo 1__: Probabilidad de Refugio sabiendo que si Ciudad y si Cooperativo. 

In [162]:
#Probando el recomendador con el ejemplo 1
vr = recomendador(Modelo_residentevil, ['Refugio'], {'Ciudad': 1, 'Cooperativo': 1})
print('La variable recomendada es:',vr)

La variable recomendada es: {'Grupo': 0}


__Ejemplo 2__: Probabilidad de Ciudad sabiendo que si Agua y no Refugio. 

In [163]:
#Probando el recomendador con el ejemplo 2
vr = recomendador(Modelo_residentevil, ['Ciudad'], {'Agua': 1, 'Refugio': 0})
print('La variable recomendada es:',vr)

La variable recomendada es: {'Lugar': 0}
